In [ ]:
from pathlib import Path
import subprocess, sys

def find_project_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / 'scripts').exists() and (p / 'outputs').exists():
            return p
    raise RuntimeError(f'Could not find project root from: {start}')

# Works when launched from repo root or from inside notebooks/
start = Path.cwd()
if start.name == 'notebooks':
    start = start.parent
PROJECT_ROOT = find_project_root(start)

DATA_DIR = PROJECT_ROOT / 'data' / 'Real-Images'
OUT_DIR = PROJECT_ROOT / 'outputs' / 'latest'
SCRIPTS_DIR = PROJECT_ROOT / 'scripts'

print('PROJECT_ROOT:', PROJECT_ROOT)
print('DATA exists:', DATA_DIR.exists())
print('TIF count:', len(list(DATA_DIR.glob('*.tif'))) if DATA_DIR.exists() else 0)
print('SCRIPTS:', sorted([p.name for p in SCRIPTS_DIR.glob('*.py')]))

In [ ]:
def run_script(script_name: str):
    script_path = SCRIPTS_DIR / script_name
    if not script_path.exists():
        raise FileNotFoundError(script_path)
    print(f'\n=== Running: {script_path.name} ===')
    # Run using the current kernel's Python
    result = subprocess.run([sys.executable, str(script_path)], cwd=str(PROJECT_ROOT), capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print(result.stderr)
    result.check_returncode()

RUN_HEAVY = True  # set False to skip long scripts

## 1) Presentation figures (fast)

In [ ]:
run_script('generate_presentation_figures_minimal.py')

## 2) Improved registration + STRidge slides (heavy)
Generates `SLIDE1..SLIDE4*.png` into `outputs/latest/slides/`.

In [ ]:
if RUN_HEAVY:
    run_script('pde_discovery_improved_registration.py')
else:
    print('Skipped (RUN_HEAVY=False)')

## 3) Model comparison (medium)
Generates `MODEL_COMPARISON.png` into `outputs/latest/slides/`.

In [ ]:
run_script('analyze_results.py')

## 4) Create simplified Slide 3 variants (fast)

In [ ]:
run_script('create_simple_slide3.py')

In [ ]:
run_script('create_method_slide.py')

## 4b) Method slide (fast)
Generates `SLIDE_METHOD_PIPELINE.png` into `outputs/latest/slides/`.

## 5) Patch-based SINDy (heavy)
Generates `patch_based_sindy_results.png` into `outputs/latest/patch_sindy/`.

## 5) Patch-based PDE discovery (robust)
Runs the robust patch-based discovery with a proper time train/test split.

Tip: start with the fixed `Model 4` term set (best global structure) and then try `full` if needed.

In [ ]:
# Best-structure patch-based (matches analyze_results.py Model 4)
if RUN_HEAVY:
    # Keep settings moderate; increase samples-per-patch for more stability
    result = subprocess.run(
        [
            sys.executable,
            str(SCRIPTS_DIR / 'patch_based_pde_discovery.py'),
            '--model','model4',
            '--max-images','51',
            '--samples-per-patch','40',
            '--patch','21',
            '--overlap','10',
            '--rt','2',
            '--rs','3',
            '--deg','3',
            '--train-frac','0.7',
        ],
        cwd=str(PROJECT_ROOT),
        capture_output=True,
        text=True,
    )
    print(result.stdout)
    if result.returncode != 0:
        print(result.stderr)
    result.check_returncode()
else:
    print('Skipped (RUN_HEAVY=False)')

In [ ]:
if RUN_HEAVY:
    run_script('patch_based_sindy.py')
else:
    print('Skipped (RUN_HEAVY=False)')

## 6) Transport-form experiment (heavy)
This currently runs but yields very low R² in the implemented LASSO setup.

In [ ]:
if RUN_HEAVY:
    run_script('transport_pde_discovery.py')
else:
    print('Skipped (RUN_HEAVY=False)')

## 7) Inspect latest outputs

In [ ]:
from datetime import datetime

paths = sorted([p for p in OUT_DIR.rglob('*.png')], key=lambda p: p.stat().st_mtime, reverse=True)
print('Most recent PNG outputs:')
for p in paths[:20]:
    ts = datetime.fromtimestamp(p.stat().st_mtime)
    print(f'- {p.relative_to(PROJECT_ROOT)}  ({ts:%Y-%m-%d %H:%M:%S})')